In [1]:
import cv2
import numpy as np
from PIL import Image
import os
import random
from torchvision import transforms


In [2]:
def flip_vertical(image):
    return cv2.flip(image, 1)

In [3]:
def rotate_image(image, angle):
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
    return cv2.warpAffine(image, matrix, (w, h))


In [4]:
def crop_image(image, side):
    h, w = image.shape[:2]
    if side == "left":
        return image[:, w//10:]  # Обрезка слева
    elif side == "right":
        return image[:, :-w//10]  # Обрезка справа
    elif side == "top":
        return image[h//10:, :]  # Обрезка сверху
    elif side == "bottom":
        return image[:-h//10, :]  # Обрезка снизу
    return image


In [5]:
def apply_blur(image, kernel_size):
    kernel_size = (3, 3)  # Убедитесь, что значения ширины и высоты нечетные.
    return cv2.GaussianBlur(image, kernel_size, 0)


In [6]:
def add_noise(image):
    noise = np.random.normal(0, 25, image.shape).astype(np.uint8)
    return cv2.add(image, noise)


In [7]:
def add_scratch(image):
    h, w = image.shape[:2]
    num_lines = random.randint(1, 5)
    for _ in range(num_lines):
        x1, y1 = random.randint(0, w), random.randint(0, h)
        x2, y2 = random.randint(0, w), random.randint(0, h)
        color = (255, 255, 255) if random.random() > 0.5 else (0, 0, 0)
        thickness = random.randint(1, 3)
        cv2.line(image, (x1, y1), (x2, y2), color, thickness)
    return image


In [8]:
def process_and_save_images(input_dir, output_dir):
    angles = [0, -10, -15, -20, 10, 15, 20]
    crops = ["left", "right", "top", "bottom"]
    blur_kernels = [0, 8, 15]

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for filename in os.listdir(input_dir):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            img_path = os.path.join(input_dir, filename)
            image = cv2.imread(img_path)
            image = cv2.resize(image, (224, 224))

            for flip in [True, False]:
                flipped_img = flip_vertical(image) if flip else image
                for angle in angles:
                    rotated_img = rotate_image(flipped_img, angle)
                    for crop in crops:
                        cropped_img = crop_image(rotated_img, crop)
                        for blur in blur_kernels:
                            blurred_img = apply_blur(cropped_img, blur) if blur > 0 else cropped_img
                            for noise in [True, False]:
                                noised_img = add_noise(blurred_img) if noise else blurred_img
                                for scratch in [True, False]:
                                    final_img = add_scratch(noised_img) if scratch else noised_img

                                    output_path = os.path.join(output_dir, f"aug_{filename.split('.')[0]}_{flip}_{angle}_{crop}_{blur}_{noise}_{scratch}.jpg")
                                    cv2.imwrite(output_path, final_img)


In [9]:
input_directory = "./input"  # Папка с исходными изображениями
output_directory = "./augmented_images"  # Папка для сохранения аугментированных изображений

process_and_save_images(input_directory, output_directory)
